# Federal awards

This notebook analyzes federal awards data (assistance prime transactions) downloaded from [USAspending.gov](https://www.usaspending.gov/search/?hash=966f5f0446cbe09c05fb1d16dee2885a&section=map).

The objective is to find which agencies awarded funding to congressional districts 14 and 15 and certain neighborhoods in the South Bronx.

In [1]:
# importing libraries
import pandas as pd

In [2]:
df = pd.read_csv("usaspending/Assistance_PrimeTransactions_2024-11-20_H01M42S17_1.csv")
pd.set_option('display.float_format', '{:.0f}'.format) # show full value of float
pd.set_option('display.max_colwidth', None) # show full strings

/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_25527/3904420668.py:1: DtypeWarning: Columns (65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("usaspending/Assistance_PrimeTransactions_2024-11-20_H01M42S17_1.csv")


In [3]:
len(df)

138297

## Setting up our working `df` 

In [4]:
# keeping only some columns, so we don't have to work with so much
filtered_df = df[["awarding_agency_name", "federal_action_obligation",
            "total_obligated_amount", "total_outlayed_amount_for_overall_award", "action_date",
            "action_date_fiscal_year", "awarding_sub_agency_name", "program_activities_funding_this_award",
             "recipient_name", "recipient_zip_code", "prime_award_transaction_recipient_cd_original",
             "cfda_title", "assistance_type_description", "transaction_description", "business_types_description",
             "usaspending_permalink"
            ]]
len(filtered_df) # should be the same as df

138297

In [5]:
filtered_df.dtypes

awarding_agency_name                              object
federal_action_obligation                        float64
total_obligated_amount                           float64
total_outlayed_amount_for_overall_award          float64
action_date                                       object
action_date_fiscal_year                            int64
awarding_sub_agency_name                          object
program_activities_funding_this_award             object
recipient_name                                    object
recipient_zip_code                               float64
prime_award_transaction_recipient_cd_original     object
cfda_title                                        object
assistance_type_description                       object
transaction_description                           object
business_types_description                        object
usaspending_permalink                             object
dtype: object

In [6]:
# convert zip floats to int
filtered_df["recipient_zip_code"] = filtered_df["recipient_zip_code"].fillna(0).astype(int)

/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_25527/2500644147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["recipient_zip_code"] = filtered_df["recipient_zip_code"].fillna(0).astype(int)


In [7]:
# keeping only those tagged NY-14 or NY-15, which are our target congressional districts
filtered_df = filtered_df[filtered_df["prime_award_transaction_recipient_cd_original"].isin(["NY-14", "NY-15"])]
len(filtered_df)

4064

In [8]:
# target zip codes
zip_codes = [{
        "zip_num": 10451,
        "neighborhood": "Melrose"
    },
    {
        "zip_num": 10455,
        "neighborhood": "Longwood"
    },
    {
        "zip_num": 10459,
        "neighborhood": "Longwood"
    },
    {
        "zip_num": 10454,
        "neighborhood": "Mott Haven"
    },
    {
        "zip_num": 10456,
        "neighborhood": "Morrisania"
    },
    {
        "zip_num": 10474,
        "neighborhood": "Hunts Point"
    }]

# converting list of dicts to df
zip_codes = pd.DataFrame(zip_codes)

# converting target column to list
zip_list = list(zip_codes["zip_num"])
zip_list

[10451, 10455, 10459, 10454, 10456, 10474]

In [9]:
# keeping only those with the zip codes we need
filtered_df = filtered_df[filtered_df["recipient_zip_code"].isin(zip_list)]
len(filtered_df)

452

In [10]:
# merging zip details, to make it easier for analysis?

filtered_df = pd.merge(filtered_df, zip_codes, left_on="recipient_zip_code", right_on="zip_num", how='left')
filtered_df = filtered_df.drop(columns=["zip_num"])
len(filtered_df) # should be the same as previous len()

452

### Analysis

We downsized our 138297-items-long dataset into 452 targeted rows. Now, we begin our analysis. The objective is to find which agencies or areas are getting the most or least federal funding.

In [11]:
# sorting by agency
agency_outlayed_sum_df = filtered_df.groupby("awarding_agency_name")["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

agency_outlayed_sum_df

,awarding_agency_name,total_outlayed_amount_for_overall_award
0,Department of Health and Human Services,472075150
1,Department of Housing and Urban Development,253902948
2,Department of Education,3025707
3,Department of Justice,1699186
4,Social Security Administration,804621
5,Department of Labor,664273
6,Federal Communications Commission,624662
7,Department of Commerce,200000
8,Department of the Treasury,138706
9,Small Business Administration,76484


In [12]:
# counting items per agency
filtered_df["awarding_agency_name"].value_counts()

awarding_agency_name
Department of Housing and Urban Development    182
Department of Health and Human Services        100
Federal Communications Commission               56
Small Business Administration                   46
Department of Education                         32
Department of Justice                           11
National Endowment for the Arts                  8
Department of Defense                            4
Department of the Treasury                       3
National Science Foundation                      2
Department of Commerce                           2
Department of Labor                              2
Social Security Administration                   1
Institute of Museum and Library Services         1
Environmental Protection Agency                  1
National Endowment for the Humanities            1
Name: count, dtype: int64

In [13]:
# sorting by subagency
subagency_outlayed_sum_df = filtered_df.groupby(["awarding_agency_name", "awarding_sub_agency_name"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

subagency_outlayed_sum_df

,awarding_agency_name,awarding_sub_agency_name,total_outlayed_amount_for_overall_award
0,Department of Health and Human Services,Health Resources and Services Administration,395487806
1,Department of Housing and Urban Development,Assistant Secretary for Housing--Federal Housing Commissioner,252853066
2,Department of Health and Human Services,Administration for Children and Families,65136631
3,Department of Health and Human Services,Substance Abuse and Mental Health Services Administration,9754853
4,Department of Education,Department of Education,3025707
5,Department of Health and Human Services,Centers for Disease Control and Prevention,1695860
6,Department of Justice,Office of Justice Programs,1679186
7,Department of Housing and Urban Development,Assistant Secretary for Community Planning and Development,1049882
8,Social Security Administration,Social Security Administration,804621
9,Department of Labor,Employment and Training Administration,664273


In [14]:
subagency_outlayed_sum_df.to_csv("subagency_totals.csv", encoding="UTF-8", index=False)

In [15]:
# sorting by zip code 
zip_outlayed_sum_df = filtered_df.groupby(["recipient_zip_code", "neighborhood"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

zip_outlayed_sum_df

,recipient_zip_code,neighborhood,total_outlayed_amount_for_overall_award
0,10459,Longwood,444480285
1,10455,Longwood,107663975
2,10454,Mott Haven,78745589
3,10456,Morrisania,58769286
4,10451,Melrose,40403107
5,10474,Hunts Point,3218969


In [16]:
zip_outlayed_sum_df.to_csv("zip_totals.csv", encoding="UTF-8", index=False)

In [17]:
# counting items per zip code
filtered_df["recipient_zip_code"].value_counts()

recipient_zip_code
10459    122
10454     80
10451     78
10456     77
10455     77
10474     18
Name: count, dtype: int64

In [18]:
# sorting by zip code and agency
zip_agency_outlayed_sum_df = filtered_df.groupby(["recipient_zip_code", "awarding_agency_name"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

zip_agency_outlayed_sum_df

,recipient_zip_code,awarding_agency_name,total_outlayed_amount_for_overall_award
0,10459,Department of Health and Human Services,378433988
1,10455,Department of Housing and Urban Development,90901620
2,10459,Department of Housing and Urban Development,66027471
3,10454,Department of Health and Human Services,46572676
4,10456,Department of Housing and Urban Development,33762017
5,10454,Department of Housing and Urban Development,31574391
6,10451,Department of Housing and Urban Development,28447254
7,10456,Department of Health and Human Services,24829214
8,10455,Department of Health and Human Services,12290237
9,10451,Department of Health and Human Services,9949035


In [19]:
zip_agency_outlayed_sum_df.to_csv("zip_agency_totals.csv", encoding="UTF-8", index=False)

In [20]:
# sorting by business type
biztype_sum_df = filtered_df.groupby("business_types_description")["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

biztype_sum_df

,business_types_description,total_outlayed_amount_for_overall_award
0,OTHER,366006416
1,FOR-PROFIT ORGANIZATION (OTHER THAN SMALL BUSINESS),251160019
2,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),110893116
3,NONPROFIT WITHOUT 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),3577582
4,PUBLIC/STATE CONTROLLED INSTITUTION OF HIGHER EDUCATION,1507868
5,INDIVIDUAL,51949
6,INDEPENDENT SCHOOL DISTRICT;NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),48888
7,SMALL BUSINESS,24535
8,INDEPENDENT SCHOOL DISTRICT,10840
9,CITY OR TOWNSHIP GOVERNMENT,0


In [21]:
# sorting by agency and business type
agency_biztype_sum_df = filtered_df.groupby(["awarding_agency_name", "business_types_description"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

agency_biztype_sum_df

,awarding_agency_name,business_types_description,total_outlayed_amount_for_overall_award
0,Department of Health and Human Services,OTHER,361621775
1,Department of Housing and Urban Development,FOR-PROFIT ORGANIZATION (OTHER THAN SMALL BUSINESS),251160019
2,Department of Health and Human Services,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),106875794
3,Department of Health and Human Services,NONPROFIT WITHOUT 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),3577582
4,Department of Education,OTHER,2322460
5,Department of Justice,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),1699186
6,Department of Housing and Urban Development,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN INSTITUTION OF HIGHER EDUCATION),1693047
7,Department of Housing and Urban Development,OTHER,1049882
8,Social Security Administration,PUBLIC/STATE CONTROLLED INSTITUTION OF HIGHER EDUCATION,804621
9,Department of Education,PUBLIC/STATE CONTROLLED INSTITUTION OF HIGHER EDUCATION,703247


In [22]:
# sorting by agency and recipient
agency_recipient_sum_df = filtered_df.groupby(["awarding_agency_name", "recipient_name"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

agency_recipient_sum_df

,awarding_agency_name,recipient_name,total_outlayed_amount_for_overall_award
0,Department of Health and Human Services,"URBAN HEALTH PLAN, INC",194957876
1,Department of Health and Human Services,LA CASA DE SALUD INC,183476111
2,Department of Health and Human Services,"EAST SIDE HOUSE, INC.",46572676
3,Department of Housing and Urban Development,NEW VISION COMMUNITY REDEVELOPMENT HOUSING DEVELOPMENT FUND CORPORATION,31640391
4,Department of Housing and Urban Development,WEST FARMS ESTATES CO L.P,22627040
...,...,...,...
140,Small Business Administration,BELINDA AFRICAN CARRIBEAN MARKET LLC,0
141,Small Business Administration,BRIGHT FUTURES 1-2-3 GROUP FAMILY DAYCARE,0
142,Small Business Administration,BUUNNI COFFEE LLC,0
143,National Science Foundation,RESEARCH FOUNDATION OF THE CITY UNIVERSITY OF NEW YORK,0


In [23]:
# sorting by agency and recipient
agency_recipient_sum_df = filtered_df.groupby(["awarding_agency_name", "awarding_sub_agency_name", "recipient_name", "neighborhood"])["total_outlayed_amount_for_overall_award"]\
.sum()\
.sort_values(ascending=False)\
.reset_index()

agency_recipient_sum_df

,awarding_agency_name,awarding_sub_agency_name,recipient_name,neighborhood,total_outlayed_amount_for_overall_award
0,Department of Health and Human Services,Health Resources and Services Administration,"URBAN HEALTH PLAN, INC",Longwood,194957876
1,Department of Health and Human Services,Health Resources and Services Administration,LA CASA DE SALUD INC,Longwood,182418768
2,Department of Health and Human Services,Administration for Children and Families,"EAST SIDE HOUSE, INC.",Mott Haven,46572676
3,Department of Housing and Urban Development,Assistant Secretary for Housing--Federal Housing Commissioner,NEW VISION COMMUNITY REDEVELOPMENT HOUSING DEVELOPMENT FUND CORPORATION,Longwood,31640391
4,Department of Housing and Urban Development,Assistant Secretary for Housing--Federal Housing Commissioner,WEST FARMS ESTATES CO L.P,Longwood,22627040
...,...,...,...,...,...
149,Small Business Administration,Small Business Administration,BUUNNI COFFEE LLC,Hunts Point,0
150,Department of Justice,"Offices, Boards and Divisions","NEW YORK, CITY OF",Melrose,0
151,Department of Health and Human Services,Administration for Children and Families,PUBLIC PREPATORY NETWORK INC.,Longwood,0
152,National Endowment for the Arts,National Endowment for the Arts,YOUNG URBAN CHRISTIANS & ARTISTS INC,Melrose,0
